In [1]:
# -*-coding:utf-8 -*-
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from nptdms import TdmsFile
from scipy.signal import savgol_filter
import tifffile as tif
from scipy import interpolate
%matplotlib widget

In [2]:
# read raw .tdms file with TdmsFile function
# name example: 20200304-160513 10pN- 30nM DNAp + 0.1mg_mL_BSA #019-002
name = input('please type in the file name:' ) + '.tdms'
# path example: C:\Users\12345\Desktop
path = input('please type in the file path:')
filename = os.path.join(path, name) 

please type in the file name: 20191017-145427 150mM Na+ 5mM Mg_ 30nM DNAp + trx  + 625uM dNTPs  #015-002
please type in the file path: C:\Users\KTS260\Google Drive\Work with Matthew\Project2_Watching labelled DNAP replication\20191017-015-2-exo+pol -good +5mM Mg2+\image data


In [58]:
# access .tdms file and convert .tdms to image data
metadata = TdmsFile.read_metadata(filename)
# print(metadata.properties)
width = metadata.properties['Pixels per line']
px_size = float(metadata.properties['Scan Command.Scan Command.scanning_axes.0.pix_size_nm'])
px_dwell_time = float(metadata.properties['Scan Command.PI Fast Scan Command.pixel_dwell_time_ms'])
inter_frame_time = float(metadata.properties['Scan Command.PI Fast Scan Command.inter_frame_wait_time_ms'])/2

tdms_file = TdmsFile(filename)
time = tdms_file['Data']['Time (ms)'][:]
time = np.array([int(i) for i in time])
position = tdms_file['Data']['Actual position X (um)'][:]
position = np.array([int(i) for i in position])
height = len(time)/width

chn_r = tdms_file['Data']['Pixel ch 1'][:]
chn_r = np.array([int(i) for i in chn_r])
chn_g = tdms_file['Data']['Pixel ch 2'][:]
chn_g = np.array([int(i) for i in chn_g])
chn_b = tdms_file['Data']['Pixel ch 3'][:]
chn_b = np.array([int(i) for i in chn_b])

chn_rgb = np.vstack((chn_r, chn_g, chn_b)).T
img = chn_rgb.reshape((int(height), int(width), 3))
img = img.transpose((1, 0, 2))
img = img.astype(np.uint16)

In [4]:
# show tif image
plt.figure(filename)
plt.imshow(img.astype('uint16'),vmax = 10)
plt.xlabel('Time/px')
plt.ylabel('Position/px')
plt.show()
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
# save .tdms file to tif image
tif.imwrite(filename + '.tiff', img)

In [59]:
# Plot DNA polymerase from image data
b,g,r = cv2.split(img)
img = g

plt.figure(figsize=(10,3))
plt.imshow(img.astype('uint8'),cmap='gray',vmax = 10,aspect ="auto")
plt.xlabel("Time/px")
plt.ylabel("Position/px")
plt.title("DNA Polymerase Trace")
plt.ylim(80,20)
plt.xlim(0,900)
# plt.xlim(0,2000)
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
# read and plot the DNA trace calculated from force measurement
trace_name = input('please type in the file name:' ) + '.xlsx'
trace_path = input('please type in the file path:')
trace_file = os.path.join(trace_path, trace_name) 
trace = pd.read_excel(trace_file)
print(trace.head())

please type in the file name: 20191017-015-2
please type in the file path: C:\Users\KTS260\Google Drive\Work with Matthew\Project2_Watching labelled DNAP replication\20191017-015-2-exo+pol -good +5mM Mg2+


     time junction_position_all
0  4301.0               2.88159
1  4350.0               2.88226
2  4399.0               2.87528
3  4448.0               2.87525
4  4499.0               2.86714


In [60]:
# Tune x_offset and y_offset to get the proper alignment
# plt.figure(figsize=(8,6))
plt.xlabel("Time/px")
plt.ylabel("Position/px")
plt.title("DNA Polymerase Trace")
x_offset = 1 # this is done by align starting time (roughly 0-5s)
y_offset = 30
x_cali = 1000/(width*px_dwell_time+inter_frame_time)  # this is calculated by (1s/time per line, in this case 163.8ms)
# x_cali = 6.5
y_cali = 1000/px_size # this is calculated by (1um/pixel size, in this case pixlsize = 75nm)
time = trace['time']/1000 * x_cali + x_offset
time = time.dropna()
position = pd.to_numeric(trace['junction_position_all'], errors='coerce') 
position = position * y_cali + y_offset
position = position.dropna()

plt.plot(time,position)
# plt.ylim(25,85)
plt.show()
plt.savefig(path + '-overlap'+'.png', dpi=150)

In [49]:
print(x_cali)
print(y_cali)

7.633587786259542
13.333333333333334


In [63]:
# this step is to find out the coordinates of image from the overlapping
# firstly, to downsample time coordinates with a correction factor: alpha = (time per line in image)/ (force data acquisition frequency, in this case 50HZ)
alpha = np.rint(173/50).astype(int)
# x = time.rolling(alpha).mean()
# a = np.array(time)
# print(a.reshape(-1,3))
x = np.mean(np.array(time[:]).reshape(-1, alpha), axis=1)
x = np.rint(x).astype(int)
# x1 here traces back to time in unit of s
x1 = (x-x_offset)/x_cali

# Since the sampling rate is different between force data and image data, therefore interpolate is performed
# secondly, interpolate time coordinate to get the position coordinate
func = interpolate.interp1d(time,position,kind='slinear')
y = func(x)
y = np.rint(y).astype(int)
plt.figure(figsize=(10,3))
plt.plot(x,y)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [64]:
# this step is to plot the intensity on the axis of DNAp trace
ins = img[y,x]
plt.figure(figsize=(8,2))
plt.plot(x1,ins)
# plt.ylim(0,30)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [91]:
# sum up 7 pixels around DNAp trace
list = [y-3,y-2, y-1, y, y+1, y+2,y+3]
intensity = np.zeros((7,len(x)), dtype=int)
all_intensity = np.zeros((len(x),), dtype=int)
for i, value in enumerate(list):
    intensity[i] = img[value,x]
#     print(intensity)
    all_intensity += img[value,x]
    
# print(intensity)
# print(all_intensity)
plt.figure(figsize=(8,3))
plt.plot(x1,all_intensity)
plt.xlabel("Time/s")
plt.ylabel("Intensity/photon")
plt.title("Intensity along DNA polymerase Trajectory")
plt.tight_layout()
plt.xlim(6,150)
# plt.ylim(0,30)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [92]:
# Apply a Savitzky-Golay filter to an array.
signal = all_intensity 
signal_filter = savgol_filter(signal[:], 41, 3)
# dt = x[1]-x[0]
# signal_filter_grad = np.gradient(signal_filter, dt)
# plt.figure()
plt.plot(x1,signal_filter)
plt.ylim(10,60)
plt.show()
plt.savefig(path + '-Intensity along DNA polymerase Trajectory'+'.png', dpi=150)

In [93]:
# Binarize the intensity with a given threshold
threshold, upper, lower = 22, 1, 0
signal_filter_binarized = np.where(signal_filter>threshold, upper, lower)

plt.figure(figsize=(8,2))
plt.xlabel("Time/s")
plt.ylabel("Binarized Intensity")
plt.plot(x1,signal_filter_binarized)
plt.ylim(-0.1,1.5)
plt.xlim(6,150)
plt.title("Intensity along DNA polymerase Trajectory")
plt.tight_layout()
plt.show()
plt.savefig(path + '-Intensity along DNA polymerase Trajectory-binarized'+'.png', dpi=150)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [74]:
plt.figure(figsize=(10,1))
# print(image)
# plt.imshow(intensity.astype('uint8'),aspect ="auto")
plt.imshow(intensity.astype('uint8'),cmap='gray',vmax = 10,interpolation='nearest', aspect='auto')

plt.axis('off')
# set_xlim(0, 150)
plt.tight_layout()
# plt.ylim(0,7)
plt.savefig(path + '-Squeezed DNAp Trace'+'.png', dpi=150)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [83]:
# This final step is to get a double x-axis figure, which shows both time in unit of pixel and s
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA

plt.figure(figsize = (10, 3))

#Setup a host axis
host = host_subplot(111, axes_class=AA.Axes)
plt.subplots_adjust(bottom=0.2)

#Add twin y axis with offset from bottom and hide top axis
par = host.twiny()
offset = -30
new_fixed_axis = host.get_grid_helper().new_fixed_axis
par.axis["bottom"] = new_fixed_axis(loc="bottom",
                                    axes=par,
                                    offset=(0, offset))

par.axis["top"].set_visible(False)
host.axis["right"].set_visible(False)

# Plot data and add main axis labels/limits
host.imshow(img.astype('uint8'),cmap='gray',vmax = 10,aspect ="auto")

host.set_xlabel("Time(pixel)")
host.set_ylabel("Position(pixel)")
host.set_title("DNA Polymerase Trace")
host.set_xlim((0,900))
host.set_ylim((80,20))

# plt.axis("tight")

#Set limits of parasite axis 
#(note this axis is not attached to 
# an actual plot so changes nothing)

par.set_xlabel("Time(s)")
par.set_xlim((x1[0],x1[-1]))

plt.plot(time,position,'y')

plt.show()
plt.tight_layout()
plt.savefig(path + '-DNA Polymerase Trace'+'.png', dpi=150)

<ipython-input-83-f7b5a44a36e8>:5: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize = (10, 3))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …